In [1]:
# Installing required libraries
!pip install numpy
!pip install pandas
!pip install torch
!pip install transformers
!pip install sentence_transformers
!pip install rouge-score
!pip install rouge
!pip install simplet5


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 21.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.16.2
    Uninstalling transformers-4.16.2:
      Successfully uninstalled transformers-4.16.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
simplet5 0.1.4 requires transformers==4.16.2, but you have transformers 4.42.3 which is incompatible.
  Using cached transformers-4.16.2-py3-none-any.whl (3.5 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.3
    Uninstalling transformers-4.42.3:
      Successfully uninstalled transformers-4.42.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.0.1 requires transformers<

In [2]:
#Importing all the libraries required
import re
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from simplet5 import SimpleT5
from sklearn.model_selection import train_test_split
from transformers import BartForConditionalGeneration, BartTokenizer
#from sentence_transformers import SentenceTransformer
from rouge import Rouge
from rouge_score import rouge_scorer

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


In [3]:
#Reading the dataset

movies = pd.read_excel('sample_data/dataset.xlsx')
movies.head()

,TITLE,Description
0,21,"Ben Campbell, a mathematics major at the Massa..."
1,10 Things I Hate About You,"Cameron James, a new student at Padua High Sch..."
2,101 Dalmatians(1996),American video game designer Roger Dearly live...
3,12 Angry Men,"On a hot summer day, a jury in the New York Co..."
4,12 Years a Slave,Solomon Northup is a free African-American man...


In [4]:
#Preprocess the text with first splitting the sentence into seperate words and set a maximum number of 1024 words
def preprocess_text(text, max_words=1024):
    if isinstance(text, float) and pd.isnull(text):
        return ''

    # Split text into words and limit to max_words
    words = text.split()[:max_words]

    # Join words back into a single string
    text = ' '.join(words)

    # Remove newline, carriage return, and tab characters
    regex = re.compile(r'[\n\r\t]')
    text = regex.sub(" ", text)

    # Remove extra spaces
    text = re.sub(' +', ' ', text).strip()

    return text

In [5]:
movies['Description'] = movies['Description'].apply(preprocess_text)

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [7]:
#Splitting data for training
plots = []
for i in movies.Description:
    try:
        k = i.split(" ")
        if 1110 > len(k) > 591:
            plots.append(i)
    except:
        continue

In [8]:
plot_list = movies.Description.tolist()

def split_list(lst, batch):
    for i in range(0, len(lst), batch):
        yield lst[i:i + batch]

batch = 32
batches = list(split_list(plot_list, batch))

In [9]:
#BART model
from transformers import BartForConditionalGeneration, BartTokenizer
model = BartForConditionalGeneration.from_pretrained('sshleifer/distilbart-cnn-12-3')
model.to(device)
model.eval()
tokenizer = BartTokenizer.from_pretrained('sshleifer/distilbart-cnn-12-3')

Downloading:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/973M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/238 [00:00<?, ?B/s]

In [10]:
#Generate summaries using BART
all_summaries = []
for batch in batches:
    with torch.no_grad():
        inputs = tokenizer(batch,
                           max_length=1024,
                           padding=True,
                           truncation=True,
                           return_tensors='pt')

        summary_ids = model.generate(inputs['input_ids'].to(device), max_length=500, early_stopping=True).to('cpu')

        all_summaries+= [txt.strip() for txt in tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)]
        del inputs, summary_ids
        torch.cuda.empty_cache()
movies['Plot Summary(bart)'] = all_summaries

In [11]:
#Save generated summaries into an excel file
movies=movies.to_excel('generated_summaries.xlsx', index=False)

In [12]:
#Display
op=pd.read_excel("generated_summaries.xlsx")
op.head()

,TITLE,Description,Plot Summary(bart)
0,21,"Ben Campbell, a mathematics major at the Massa...","Ben Campbell, a mathematics major at the Massa..."
1,10 Things I Hate About You,"Cameron James, a new student at Padua High Sch...",Michael Eckman warns sophomore Bianca Stratfor...
2,101 Dalmatians(1996),American video game designer Roger Dearly live...,American video game designer Roger Dearly live...
3,12 Angry Men,"On a hot summer day, a jury in the New York Co...",The 18-year-old boy is accused of killing his ...
4,12 Years a Slave,Solomon Northup is a free African-American man...,Slave trader Theophilus Freeman gives Northup ...


In [13]:
#Install required libraries
!pip install numpy pandas faiss-gpu torch transformers sentence_transformers rouge --quiet
!pip install rouge-score
!pip install wikipedia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
simplet5 0.1.4 requires transformers==4.16.2, but you have transformers 4.42.3 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11694 sha256=54185728a0fea86b95c29981ae79b55eac47dbd236dd660f0153369ea4e1c25a
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [16]:
#Import required libraries
import re
import numpy as np
import pandas as pd
import faiss
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from transformers import BartTokenizer, BartForConditionalGeneration
from rouge import Rouge
from rouge_score import rouge_scorer

In [17]:
#Generate summaries for movies not in the dataset by referring to wikipedia and using t5 model
import wikipedia
import requests
from bs4 import BeautifulSoup
from transformers import T5Tokenizer, T5ForConditionalGeneration
import nltk
from nltk.tokenize import sent_tokenize

# Initialize NLTK tokenizer and download punkt if necessary
nltk.download('punkt')

# Initialize T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Initialize Rouge
rouge = Rouge()

def fetch_movie_plot(movie_name):
    try:
        # Directly fetch the page for the movie
        page = wikipedia.page(movie_name)
        content = page.content

        # Find the plot section and extract the content
        plot_section = ""
        start_idx = content.find("== Plot ==")

        if start_idx == -1:
            start_idx = content.find("== Plot==")  # Check for variations in section header

        if start_idx != -1:
            end_idx = content.find("\n==", start_idx + len("== Plot =="))
            if end_idx == -1:
                plot_section = content[start_idx:].strip()  # If end of section not found, take until end of content
            else:
                plot_section = content[start_idx:end_idx].strip()
        else:
            plot_section = content

        return plot_section

    except wikipedia.exceptions.PageError:
        return f"Page not found for '{movie_name}'. Please check the title and try again."
    except wikipedia.exceptions.DisambiguationError as e:
        # If there are multiple options, you can handle it here (e.g., select the first one)
        try:
            page = wikipedia.page(e.options[0])
            content = page.content

            plot_section = ""
            start_idx = content.find("== Plot ==")

            if start_idx == -1:
                start_idx = content.find("== Plot==")  # Check for variations in section header

            if start_idx != -1:
                end_idx = content.find("\n==", start_idx + len("== Plot =="))
                if end_idx == -1:
                    plot_section = content[start_idx:].strip()  # If end of section not found, take until end of content
                else:
                    plot_section = content[start_idx:end_idx].strip()
            else:
                plot_section = content

            return plot_section

        except wikipedia.exceptions.PageError:
            return f"Page not found for '{movie_name}'. Please check the title and try again."
        except wikipedia.exceptions.DisambiguationError:
            return f"Disambiguation error encountered for '{movie_name}'. Please refine your search."


def generate_summary(content, max_length=1024):
    if content and not content.startswith("Page not found"):
        # Tokenize the content into sentences using NLTK
        sentences = sent_tokenize(content)

        # Generate summary using T5
        inputs = tokenizer.encode("summarize: " + content, return_tensors="pt", max_length=1024, truncation=True)
        summary_ids = model.generate(inputs, max_length=900, min_length=100, length_penalty=5.0, num_beams=10)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        # Extract unique sentences from the generated summary
        unique_summary = []
        for sentence in sent_tokenize(summary):
            if sentence not in unique_summary:
                unique_summary.append(sentence)

        # Join unique sentences back into a summary
        final_summary = '. '.join(unique_summary)

        return final_summary
    else:
        return content

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [25]:
df = pd.read_excel("sample_data/generated_summaries.xlsx")
print(df.columns)

Index(['TITLE', 'Description', 'Plot Summary(bart)'], dtype='object')


In [30]:
import pandas as pd
from rouge import Rouge
import nltk
from nltk.tokenize import word_tokenize
import wikipedia  # Import Wikipedia module

# Initialize NLTK tokenizer and download punkt if necessary
nltk.download('punkt')

# Initialize ROUGE
rouge = Rouge()

# Read your Excel file into DataFrame
df = pd.read_excel("sample_data/generated_summaries.xlsx")

# Function to fetch movie plot summary from Wikipedia
def fetch_movie_plot(movie_title):
    try:
        page = wikipedia.page(movie_title)
        return page.content
    except wikipedia.exceptions.PageError:
        return None

# Function to generate summary using a placeholder function
def generate_summary(content):
    # Placeholder function; replace with your actual summarization method
    return "Generated summary: " + content[:100]

# Function to get plot summary
def get_plot_summary(movie_title, df):
    summary = df.loc[df['TITLE'] == movie_title, 'Plot Summary(bart)']
    if not summary.empty:
        content = df.loc[df['TITLE'] == movie_title, 'Description'].values[0]
        return summary.values[0], content  # Return both summary and content
    else:
        wikipedia_summary = fetch_movie_plot(movie_title)
        if wikipedia_summary:
            t5_summary = generate_summary(wikipedia_summary)
            return t5_summary, wikipedia_summary
        else:
            return f"Plot summary for '{movie_title}' not found in dataset or Wikipedia.", None

# Main loop
while True:
    movie_title = input("\nEnter title of the movie to summarize (type 'exit' to quit):\n")
    if movie_title.lower() == 'exit':
        print("\nExited")
        break
    else:
        summary, content = get_plot_summary(movie_title, df)
        print(summary)

        if content and summary:
            # Calculate ROUGE-L scores
            rouge_scores = rouge.get_scores(summary, content, avg=True)

            # Print ROUGE-L scores
            print("\nROUGE-L Scores:")
            print(f"Precision:", rouge_scores['rouge-l']['p'])
            print(f"Recall:", rouge_scores['rouge-l']['r'])
            print(f"F1 Score:", rouge_scores['rouge-l']['f'])
        else:
            print("\nUnable to calculate ROUGE-L scores.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



Enter title of the movie to summarize (type 'exit' to quit):
12 aNGRY mEN
Generated summary: Twelve Angry Men is an American courtroom drama written by Reginald Rose concerning the jury of a ho

ROUGE-L Scores:
Precision: 0.85
Recall: 0.019813519813519812
F1 Score: 0.038724373131106625

Enter title of the movie to summarize (type 'exit' to quit):
12 Angry Men
The 18-year-old boy is accused of killing his abusive father. A neighbor testified to witnessing the defendant stab his father, from her window, through the windows of a passing train. In a preliminary vote, all jurors vote "guilty" except Juror 8, who believes there should be some discussion before the verdict is made. Juror 3, infuriated, argues with and tries to attack Juror.

ROUGE-L Scores:
Precision: 1.0
Recall: 0.1569767441860465
F1 Score: 0.2713567815742027

Enter title of the movie to summarize (type 'exit' to quit):
zootopia
Generated summary: Zootopia (titled Zootropolis or Zoomania in various regions) is a 2016 Americ